In [1]:
import cv2
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [2]:
target_size = (1280,720)
model_input_shape = (299,299)
model1 = load_model("AutoQuest_first_model.h5")
model2 = load_model("AutoQuest_second_model.h5")
first_classes = ['dialog', 'loading', 'others', 'reward']
second_classes = ['idle', 'inQuest']

In [3]:
def read_and_resize(path,target_size):
    return cv2.resize(cv2.imread(path),target_size)/255

def crop_img(inputImg, scenario,model_input_shape):
    if scenario == 'reward':
        return inputImg[:,335:940]
    elif scenario == 'dialog':
        return inputImg[460:,]
    elif scenario == 'questArrow':
        return inputImg[460:,]
    elif scenario == 'autoIcon':
        icon = inputImg[632:698,351:416]
        icon = cv2.resize(icon,model_input_shape)
        return icon

def similarity_coef(A,B):
    return abs(np.corrcoef(A.flatten(),B.flatten())[0,1])

# input an Game image with target size and normalized with 255
def check_scenario(inputImg):
    inputImg_1 = cv2.resize(inputImg,model_input_shape)
    inputImg_1 = np.expand_dims(inputImg_1,0)
    pred = model1.predict(inputImg_1)
    first_scenario = first_classes[np.argmax(pred)]
    if first_scenario != 'others':
        print(f"Current scenario is {first_scenario}")
        return first_scenario
    else:
        inputImg_2 = crop_img(inputImg,'autoIcon',model_input_shape)
        inputImg_2 = np.expand_dims(inputImg_2,0)
        pred = model2.predict(inputImg_2)
        second_classes = ['idle', 'inQuest']
        second_scenario = second_classes[np.argmax(pred)]
        print(f"Current scenario is {second_scenario}")
        return second_scenario

# Run script

In [4]:
from win32gui import GetWindowText, GetForegroundWindow, GetWindowRect, FindWindow
import pyscreenshot as ImageGrab
import time
import os
from IPython.display import display, clear_output
import datetime

'''
# if the foreground window is as the same as desired_window, then this script will capture your current screen and check current scenario.
# which is faster than capture screenshot using Nox.
'''
desired_window = "夜神模擬器"

# The Nox install path
path = "C:\\Program Files\\Nox\\bin"
os.chdir(path)

'''
The simulator default port, if you using multiple simulator, you should change the port.
You can check your port number use : os.popen('adb devices').read()
'''
print(os.popen('adb devices').read())
connect = os.popen("adb connect 127.0.0.1:62001").read()
print(connect)

'already connected to 127.0.0.1:62001\n'

In [19]:
run_count = 0
while True:
    clear_output(wait=True)
    run_count += 1
    print(f"Current round count:{run_count}")
    if run_count%1000==0:
        #restart adb server and connection after a while
        os.system("adb shell kill-server")
        os.system("adb shell start-server")
        os.popen("adb connect 127.0.0.1:62001")
    print(datetime.datetime.now())
    s = time.time()
    current_window = GetWindowText(GetForegroundWindow())
    
    # If current_window is not as desired_window `夜神模擬器`, capture screen use adb tools
    if current_window != desired_window:
        print("ForegroundWindow not Nox, adb screencap is using.")
        os.system('adb shell screencap -p /sdcard/screenshot.png')
        os.system('adb pull /sdcard/screenshot.png ./screenshot.png')
        im = cv2.imread('./screenshot.png')
        os.system('adb shell rm /sdcard/screenshot.png')
        os.remove('./screenshot.png')
    else:
        rect = GetWindowRect(FindWindow(None, desired_window))
        im = ImageGrab.grab(bbox=(rect[0],rect[1]+35,rect[2],rect[3]))
        im = np.array(im)
    im = cv2.resize(im,target_size)/255
    current_scenario = check_scenario(im)
    print(f"Computing time : {round(time.time() - s,3)} second.")
    if current_scenario == 'dialog':
        print("dialog")
        os.system("adb shell input tap 515 682")
        os.system("adb shell input tap 1377 521")
        time.sleep(2)
    elif current_scenario == 'reward':
        print("reward")
        os.system("adb shell input tap 795 795")
        time.sleep(2)
    elif current_scenario == 'idle':
        print("idle")
        os.system("adb shell input tap 134 294")
        time.sleep(2)
    elif current_scenario == 'inQuest':
        time.sleep(10)
    else:
        time.sleep(2)
    

Current round count:20
2019-10-28 17:34:53.594796
Current scenario is reward
Computing time : 0.356 second.
reward


KeyboardInterrupt: 